# La divergenza di Kullback-Leibler {#sec-kl}

**Prerequisiti**

**Concetti e competenze chiave**

- Comprendere il concetto di divervenza di Kullback-Leibler ($D_{\text{KL}}$).
- Calcolare la divergenza $D_{\text{KL}}$ dall'entropia.

**Preparazione del Notebook**

In [2]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import networkx as nx
from scipy.special import kl_div
import arviz as az
import math
from collections import Counter
from scipy.stats import entropy
import warnings
warnings.filterwarnings("ignore")

In [3]:
%config InlineBackend.figure_format = 'retina'
RANDOM_SEED = 8927
rng = np.random.default_rng(RANDOM_SEED)
az.style.use("arviz-darkgrid")

## Introduzione {.unnumbered}

Nel campo della statistica bayesiana, è fondamentale confrontare diversi modelli predittivi per determinare quale si adatta meglio ai dati disponibili. In questo capitolo, analizzeremo come l'entropia può essere utilizzata per valutare la "distanza" tra un modello teorico e i dati osservati, introducendo la divergenza di Kullback-Leibler ($D_{\text{KL}}$). Questa misura quantifica le discrepanze tra due distribuzioni di probabilità, fornendo un indicatore di quanto accuratamente un modello rappresenti le osservazioni empiriche.

## La Generalizzabilità dei Modelli e il Metodo Scientifico

La generalizzabilità dei modelli è un concetto fondamentale nella scienza e uno dei pilastri del metodo scientifico. Questo concetto si riferisce alla capacità di un modello di essere applicato con successo e di produrre risultati accurati al di fuori del contesto specifico o del set di dati per cui è stato inizialmente sviluppato o testato. In altre parole, il valore scientifico di un modello dipende fortemente dalla sua capacità di generalizzare a nuovi dati.

Nella pratica, la generalizzabilità di un modello può essere compromessa da due problemi principali: il sotto-adattamento e il sovra-adattamento. 

- **Sotto-adattamento**: Si verifica quando un modello è troppo semplice per rappresentare adeguatamente la complessità dei dati. Un modello sotto-adattato non riesce a catturare le relazioni essenziali nei dati, portando a prestazioni scadenti sia sul set di dati di addestramento che su nuovi dati. Questo limita gravemente la sua utilità in applicazioni pratiche.

- **Sovra-adattamento**: Si manifesta quando un modello è eccessivamente complesso e si adatta troppo fedelmente al rumore o alle particolarità specifiche del set di dati di addestramento. Un modello sovra-adattato può mostrare ottime prestazioni sui dati di addestramento, ma fallisce nel generalizzare a nuovi dati, riducendo la sua efficacia predittiva.

L'approccio bayesiano alla modellazione offre un modo efficace per bilanciare la complessità del modello con l'adattamento ai dati. Come descritto da @McElreath_rethinking, la selezione del modello è un processo che richiede un equilibrio tra la semplicità del modello e la sua capacità di rappresentare accuratamente la realtà dei dati.

Una pratica comune nella scelta tra modelli alternativi si basa sul principio del rasoio di Ockham, che favorisce le spiegazioni più semplici quando ci sono più teorie equivalenti per un fenomeno. Tuttavia, questo principio da solo non è sufficiente; è fondamentale che il modello scelto descriva accuratamente i dati.

Tradizionalmente, la selezione dei modelli è stata spesso basata sull'uso dei valori-p, ma, come evidenziato da @McElreath_rethinking, questo approccio presenta numerosi problemi e manca di una giustificazione teorica solida. 

Un metodo più robusto e teoricamente fondato è l'uso della divergenza di Kullback-Leibler, che misura quanto un modello riesca ad approssimare efficacemente la distribuzione reale dei dati. Questa misura fornisce una stima quantitativa della precisione del modello nel rappresentare il processo generativo sottostante. Questo capitolo introduce il concetto di divergenza di Kullback-Leibler e le sue applicazioni nella selezione dei modelli. 

## L'Entropia Relativa

Nel @sec-entropy abbiamo illustrato la relazione tra l'entropia e la lunghezza media minima del codice binario necessario per descrivere la distribuzione di una variabile casuale. Possiamo ora introdurre il concetto di *entropia relativa*, conosciuta anche come *divergenza di Kullback-Leibler*  [@kullback1951information]. L'entropia relativa è una misura della "distanza" tra due distribuzioni di probabilità, che quantifica quanto una distribuzione si discosta da un'altra.

### Concetto di Divergenza di Kullback-Leibler

La *divergenza di Kullback-Leibler*, indicata come $D(P \parallel Q)$, è una misura dell'inefficienza che si verifica assumendo che i dati seguano una distribuzione $Q$, quando in realtà la distribuzione corretta è $P$. In altre parole, questa misura quantifica la perdita di efficienza nel codificare dati se si utilizza una distribuzione errata anziché quella reale.

Dal punto di vista della teoria dell'informazione, $D(P \parallel Q)$ esprime quanto sia meno efficiente, in termini di lunghezza media del codice, utilizzare una distribuzione errata $Q$ per descrivere i dati, rispetto a usare la distribuzione vera $P$. 

Se conoscessimo la distribuzione vera $P$ di una variabile casuale, potremmo costruire un codice ottimale basato sulla codifica di Huffman, con una lunghezza media pari all'entropia $H(P)$. Tuttavia, se si costruisce un codice sulla base di una distribuzione diversa $Q$, la lunghezza media richiesta per descrivere la variabile casuale aumenterebbe. In questo caso, la lunghezza media del codice sarebbe pari a $H(P) + D(P \parallel Q)$ bit. 

La quantità $D(P \parallel Q)$ rappresenta quindi il sovraccarico, o l'inefficienza aggiuntiva, derivante dall'uso di $Q$ al posto di $P$. Più la distribuzione $Q$ differisce da $P$, maggiore sarà la divergenza di Kullback-Leibler, e quindi più inefficiente sarà la codifica basata su $Q$. 

### Definizione della Divergenza di Kullback-Leibler

Per una variabile casuale discreta, la divergenza di Kullback-Leibler è definita come:

$$
D(P \parallel Q) = \sum_{x} p(x) \log_2 \left( \frac{p(x)}{q(x)} \right),
$$ {#eq-kl-def}

dove:

- $p(x)$ è la probabilità associata al valore $x$ secondo la distribuzione vera $P$.
- $q(x)$ è la probabilità associata al valore $x$ secondo la distribuzione approssimata $Q$.

Questa formula rappresenta la somma, pesata dalle probabilità $p(x)$, dei logaritmi del rapporto tra le probabilità delle due distribuzioni.

### Entropia, Entropia Incrociata e Divergenza KL

Per chiarire ulteriormente il concetto, mostriamo che la divergenza di Kullback-Leibler può essere vista come la differenza tra l'entropia incrociata tra $P$ e $Q$, e l'entropia di $P$:
   
$$
D_{\text{KL}}(P \parallel Q) = H(P, Q) - H(P),
$$ {#eq-kl-difference}

laddove l'entropia incrociata $H(P, Q)$ misura l'incertezza media quando si usa la distribuzione $Q$ per descrivere la variabile casuale $X$ distribuita secondo $P$:
   
$$
H(P, Q) = -\sum_x p(x) \log_2(q(x)).
$$ {#eq-cross-entropy}

L'entropia incrociata rappresenta la sorpresa media se si assume che i dati siano distribuiti secondo $Q$ quando in realtà seguono $P$.

Sostituendo le espressioni per $H(P)$ e $H(P, Q)$ nella definizione della divergenza di Kullback-Leibler, otteniamo:

$$
D_{\text{KL}}(P \parallel Q) = \left(- \sum_x p(x) \log_2 q(x)\right) - \left(- \sum_x p(x) \log_2 p(x)\right).
$$

Notiamo che i segni negativi possono essere eliminati, portando a:

$$
D_{\text{KL}}(P \parallel Q) = \sum_x p(x) \log_2 p(x) - \sum_x p(x) \log_2 q(x).
$$

Ora possiamo raccogliere i due termini in una singola somma:

$$
D_{\text{KL}}(P \parallel Q) = \sum_x p(x) \left( \log_2 p(x) - \log_2 q(x) \right).
$$

Utilizziamo la proprietà dei logaritmi $\log_2 \left(\frac{a}{b}\right) = \log_2 a - \log_2 b$ per combinare i termini all'interno del logaritmo e otteniamo la formula esplicita della divergenza di Kullback-Leibler:

$$
D_{\text{KL}}(P \parallel Q) = \sum_x p(x) \log_2 \left( \frac{p(x)}{q(x)} \right).
$$

Questa espressione quantifica la differenza tra le distribuzioni $P$ e $Q$, misurando l'inefficienza nell'uso di $Q$ per rappresentare $P$.

### Interpretazione della Divergenza KL

La divergenza $D_{\text{KL}}(P \parallel Q)$ rappresenta il "costo" aggiuntivo di sorpresa o inefficienza quando si utilizza la distribuzione $Q$ per modellare i dati che in realtà seguono la distribuzione $P$. Questo "costo" è espresso in bit e rappresenta l'informazione che viene "persa" quando $Q$ è usata al posto di $P$.

È importante notare che la divergenza di Kullback-Leibler è asimmetrica, il che significa che $D(P \parallel Q)$ non è uguale a $D(Q \parallel P)$, e non può essere interpretata come una vera e propria "distanza" tra le distribuzioni, ma piuttosto come una misura dell'informazione persa, ovvero di inefficienza di codifica.

In sintesi, l'entropia relativa o divergenza di Kullback-Leibler è una misura chiave nella teoria dell'informazione. Essa valuta l'efficacia di un modello probabilistico confrontando una distribuzione teorica con la distribuzione vera dei dati. La divergenza KL fornisce un metodo per quantificare la perdita di informazione e l'incremento di incertezza quando si utilizza una distribuzione approssimativa per descrivere dati reali.

::: {#exm-}

Supponiamo che $P$ e $Q$ siano due distribuzioni di probabilità su un insieme finito di possibili esiti, ad esempio {0, 1, 2}. Consideriamo che $P$ e $Q$ siano definite come segue:

- $P$ è la distribuzione "vera": $P = [0.1, 0.6, 0.3]$;
- $Q$ è una distribuzione alternativa che usiamo per la stima: $Q = [0.2, 0.5, 0.3]$.

In [12]:
# Definizione delle distribuzioni
P = np.array([0.1, 0.6, 0.3])
Q = np.array([0.2, 0.5, 0.3])

# Calcolo della divergenza KL da P a Q
KL_divergence = np.sum(kl_div(P, Q))

print(f"Divergenza KL da P a Q: {KL_divergence:.4f}")

Divergenza KL da P a Q: 0.0401


Nel codice precedente, `kl_div(P, Q)` calcola la divergenza $D_{\text{KL}}$ elemento per elemento dell'array. Essa calcola $\sum_x p(x) \log \left(\frac{p(x)}{q(x)}\right)$ per ogni esito $x$, che è esattamente il termine $p(x) \log \left(\frac{p(x)}{q(x)}\right)$ descritto nella formula della divergenza $D_{\text{KL}}$. Utilizziamo poi `np.sum` per sommare tutti i contributi individuali e ottenere il valore totale della divergenza $D_{\text{KL}}$.

Questo esempio fornisce un calcolo diretto della divergenza $D_{\text{KL}}$ tra due distribuzioni, mostrando come una distribuzione $Q$ possa essere inadeguata nel modellare una distribuzione $P$, con un focus sul "costo" di sorpresa per ogni esito.
:::

::: {#exm-}
In altri due esempi, rendiamo via via $Q$ più diverso da $P$. Notiamo come la divergenza $D_{\text{KL}}$ aumenta.

In [13]:
P = np.array([0.1, 0.6, 0.3])
Q = np.array([0.35, 0.3, 0.35])
KL_divergence = np.sum(kl_div(P, Q))
print(f"Divergenza KL da P a Q: {KL_divergence:.4f}")

Divergenza KL da P a Q: 0.2444


In [14]:
P = np.array([0.1, 0.6, 0.3])
Q = np.array([0.6, 0.3, 0.1])
KL_divergence = np.sum(kl_div(P, Q))
print(f"Divergenza KL da P a Q: {KL_divergence:.4f}")

Divergenza KL da P a Q: 0.5663


:::

## Applicazione della Divergenza $D_{\text{KL}}$ nella Selezione di Modelli

La divergenza di Kullback-Leibler, $D_{\text{KL}}$, è uno strumento fondamentale nella selezione dei modelli statistici. L'obiettivo è identificare il modello $Q$ che minimizza $D_{\text{KL}}(P \parallel Q)$, riducendo al minimo la differenza tra l'entropia della distribuzione vera $P$ e l'entropia incrociata tra $P$ e $Q$. In altre parole, si cerca di minimizzare l'errore introdotto nell'approssimare la distribuzione reale $P$ con il modello $Q$.

### Proprietà Chiave

- **Non-negatività:** La divergenza $D_{\text{KL}}(P \parallel Q)$ è sempre maggiore o uguale a zero. Questo valore è pari a zero solo quando le distribuzioni $P$ e $Q$ sono identiche, indicando una perfetta corrispondenza tra il modello e la distribuzione vera.
  
- **Asimmetria:** $D_{\text{KL}}(P \parallel Q) \neq D_{\text{KL}}(Q \parallel P)$. Questa proprietà mostra che la "distanza" percepita tra $P$ e $Q$ dipende dalla direzione in cui è misurata. La divergenza di $Q$ rispetto a $P$ non è la stessa della divergenza di $P$ rispetto a $Q$.

### Selezione dei Modelli Statistici

Nel contesto della selezione dei modelli statistici, l'obiettivo principale è scegliere il modello $Q$ che minimizzi la divergenza $D_{\text{KL}}(P \parallel Q)$ rispetto alla distribuzione reale $P$ dei dati. Tuttavia, poiché la distribuzione vera $P$ è spesso sconosciuta o non direttamente osservabile, non è possibile calcolare la divergenza $D_{\text{KL}}$ in modo diretto.

Per superare questa limitazione, i ricercatori e gli statistici utilizzano criteri approssimativi per stimare indirettamente la divergenza $D_{\text{KL}}$. Questi criteri considerano sia la bontà di adattamento del modello ai dati osservati sia la complessità del modello stesso, cercando un equilibrio tra accuratezza e parsimonia. Nel capitolo successivo, esploreremo questi criteri in dettaglio e discuteremo come vengono utilizzati per valutare e selezionare i modelli statistici migliori.

## Riflessioni Conclusive

In questo capitolo, abbiamo affrontato la questione di come l'entropia possa essere impiegata per valutare la "distanza" tra un modello teorico e i dati reali. A tale scopo, abbiamo introdotto la divergenza $\mathbb{KL}$, una misura che quantifica le discrepanze tra due distribuzioni di probabilità. 

Nel capitolo successivo, approfondiremo ulteriormente il tema della divergenza $\mathbb{KL}$. Esploreremo come questo strumento possa essere utilizzato per confrontare modelli teorici con dati empirici e ci concentreremo su come possa fornirci una comprensione più dettagliata dell'adattamento di un modello alla realtà che intende rappresentare. Questa esplorazione ci permetterà di valutare più accuratamente la validità e la generalizzabilità dei modelli scientifici nel loro tentativo di catturare e interpretare la complessità dei fenomeni oggetto di studio. Specificamente, presenteremo la tecnica della Validazione Incrociata Leave-One-Out (LOO-CV), che viene utilizzata per calcolare un'approssimazione della divergenza $D_{\text{KL}}$. Questa tecnica consente di stimare quanto bene un modello generalizzi i dati, offrendo un'ulteriore metrica per la valutazione dei modelli statistici.

## Esercizi

::: {#exr-entropy-1}
Cosideriamo due distribuzioni di probabilità discrete, $p$ e $q$:

```
p = np.array([0.2, 0.5, 0.3])
q = np.array([0.1, 0.2, 0.7])
```

Si calcoli l'entropia di $p$, l'entropia incrociata tra $p$ e $q$, la divergenza di Kullback-Leibler da $p$ a $q$.

Si consideri `q = np.array([0.2, 0.55, 0.25])` e si calcoli di nuovo a divergenza di Kullback-Leibler da $p$ a $q$. Si confronti con il risultato precedente e si interpreti.
:::

::: {#exr-entropy-2}
Sia $p$ una distribuzione binomiale di parametri $\theta = 0.2$ e $n = 5$. Sia $q_1$ una approssimazione a $p$: `q1 = np.array([0.46, 0.42, 0.10, 0.01, 0.01])`. Sia $q_2$ una distribuzione uniforme: `q2 = [0.2] * 5`. Si calcoli la divergenza $\mathbb{KL}$ di $q_1$ da $p$ e da $q_2$ da $p$ e si interpretino i risultati.
:::

::: {#exr-entropy-3}
La Divergenza $\mathbb{KL}$ è spesso paragonata a una "distanza" tra due distribuzioni di probabilità, ma è fondamentale capire che non è simmetrica. Questo significa che la misura di quanto $p$ è diversa da $q$ non è la stessa di quanto $q$ è diversa da $p$. Questa asimmetria riflette la differenza nella perdita di informazione quando si sostituisce una distribuzione con l'altra.

Per le seguenti distribuzioni

```
p = np.array([0.01, 0.99])
q = np.array([0.7, 0.3])
```

si calcoli l'entropia di p, l'entropia incrociata da p a q, la divergenza KL da p a q, l'entropia di q, l'entropia incrociata da q a p, e la divergenza KL da q a p. Si commenti.
:::

## Informazioni sull'Ambiente di Sviluppo {.unnumbered}

In [12]:
%load_ext watermark
%watermark -n -u -v -iv -w -m

Last updated: Fri Jul 26 2024

Python implementation: CPython
Python version       : 3.12.4
IPython version      : 8.26.0

Compiler    : Clang 16.0.6 
OS          : Darwin
Release     : 23.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit

numpy     : 1.26.4
matplotlib: 3.9.1
scipy     : 1.14.0
pandas    : 2.2.2
arviz     : 0.18.0

Watermark: 2.4.3

